The main solving big ridge regression problems

In [ ]:
workspace()

In [ ]:
using Plots

In [81]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [76]:
#Basic parameters
## Basic parameters
maxiter=1000;
max_time = 100;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [77]:
dataset = "mnist.scale"   # mnist.scale  a9a  gisette_scale  rcv1_train.binary
prob = defineProb(dataset);
showDetails(dataset)

../data/mnist.scale.jld


Dict{Symbol,Any} with 3 entries:
  :sparsity => 0.192183
  :dims     => 780
  :n        => 60000

In [78]:
norm(prob.A*prob.xsol - prob.b)

2.2461272716752256e-9

In [82]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [83]:
# Testing Rademarcher Sketch
options.AUX = [1, 1]; # use sign flipping
method_name = "rademacher";# ROS
options.sketchsize =10;#Int(floor((prob.n)^(1/2)));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-10-1.0
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 23.70  |  11.30  | 0.3510 
 50  | 16.50  |  7.14  | 0.4629 
 75  | 13.11  |  5.87  | 0.5924 
100  | 10.65  |  5.65  | 0.7372 
125  | 9.14  |  4.26  | 0.8709 
150  | 7.55  |  3.95  | 1.0220 
175  | 6.39  |  3.47  | 1.1614 
200  | 5.69  |  3.22  | 1.2790 
225  | 5.52  |  2.46  | 1.3963 
250  | 4.91  |  2.60  | 1.5044 
275  | 4.01  |  2.10  | 1.6112 
300  | 4.05  |  1.92  | 1.7172 
325  | 3.77  |  1.84  | 1.8219 
350  | 3.52  |  1.74  | 1.9225 
375  | 3.29  |  1.48  | 2.0273 
400  | 3.10  |  1.48  | 2.1284 
425  | 2.44  |  1.49  | 2.2287 
450  | 2.49  |  1.29  | 2.3291 
475  | 2.63  |  1.38  | 2.4390 
500  | 2.46  |  1.14  | 2.5463 
525  | 2.33  |  1.15  | 2.6528 
550  | 2.22  |  1.17  | 2.7644 
575  | 2.11  |  1.10  | 2.8758 
600  | 2.24  |  1.11  | 2.9791 
625  | 1.35  |  1.25  | 3.0854 
650  | 1.54  |  0.81  | 3.1989 
675  | 1.84  |  0.93  | 3.3304 
700  | 1.15  |  0.83  | 3.4591 
725  

In [65]:
    
rho = Int(options.AUX[2]); # the density per row
s = options.sketchsize;
ind = sample(1:prob.n,rho*s,replace=false); # shuffle all indices
ind =reshape(ind, s,rho);
DATA = zeros(s,prob.n);

 function mapitf(A::Array{Float64}, ind, DATA)
 DATA = map( x-> sum(A[ind[x,:],:],1), 1:s)
 end

function loopitf(A::Array{Float64}, ind, DATA)
    for i =1:s
        DATA[i,:] = sum(A[ind[i,:],:],1);
    end
end

mapitf(prob.A,ind,DATA)

#loopitf(prob.A,ind,DATA)
#sum(prob.A[ind[1,:],:],1)
#DATA

1×780 Array{Float64,2}:
 -1.31195e5  -1.31195e5  -1.31195e5  …  -1.31273e5  -1.3128e5  -1.31212e5

In [84]:
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 10; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 23.50  |  11.03  | 0.1448 
 50  | 16.72  |  6.88  | 0.2610 
 75  | 12.77  |  5.86  | 0.3802 
100  | 10.27  |  5.51  | 0.4993 
125  | 8.86  |  3.86  | 0.6161 
150  | 7.77  |  3.36  | 0.7305 
175  | 6.95  |  2.91  | 0.8327 
200  | 6.24  |  2.95  | 0.9331 
225  | 5.66  |  2.68  | 1.0379 
250  | 5.18  |  2.26  | 1.1424 
275  | 4.70  |  2.32  | 1.2536 
300  | 4.84  |  2.00  | 1.3579 
325  | 4.02  |  2.06  | 1.4581 
350  | 3.72  |  1.77  | 1.5926 
375  | 3.47  |  1.86  | 1.7295 
400  | 3.19  |  1.53  | 1.8620 
425  | 3.00  |  1.50  | 1.9960 
450  | 2.85  |  1.28  | 2.1177 
475  | 2.70  |  1.31  | 2.2545 
500  | 2.36  |  1.24  | 2.3853 
525  | 2.92  |  1.11  | 2.5120 
550  | 2.31  |  1.25  | 2.6392 
575  | 2.19  |  1.24  | 2.7704 
600  | 2.10  |  0.96  | 2.8991 
625  | 2.96  |  1.06  | 3.0011 
650  | 1.03  |  1.16  | 3.1038 
675  | 1.14  |  1.12  | 3.2069 
700  | 1.86  |  0.96  | 3.3093 
725  | 1.

In [ ]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =10; # Int(floor(log2(prob.n)));
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing countmin 
options.sketchsize =10;#floor((prob.n)^(1/2));
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing CG 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];